In [55]:
import re, string
from decimal import Decimal
from pyspark.sql import Row
from math import log10 as log
from pyspark.sql.functions import desc
# from pyspark.context import SparkContext, SparkConf
# from pyspark import StorageLevel
# from pyspark import RDD

In [4]:
text_file = sc.textFile('jane_austen/pride_and_prejudice.txt')

# Se genera un RDD con id del documento como clave y el contenido como valor

In [5]:
books = sc.wholeTextFiles('jane_austen')

In [117]:
punc = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'
stopwords= ["a", "about", "above", "above", "across", "after", "afterwards" 
            , "again", "against", "all","almost", "alone", "along", "already", 
            "also","although","always","am","among", "amongst", "amoungst", 
            "amount", "an", "and", "another", "any","anyhow","anyone","anything",
            "anyway", "anywhere", "are", "around", "as",  "at","back","be","became",
            "because","become","becomes", "becoming", "been", "before", "beforehand", 
            "behind", "being","below", "beside", "besides", "between", "beyond", "bill", 
            "both", "bottom","but", "by", "call", "can", "cannot","cant", "co", "con", "could", 
            "couldnt", "cry", "de", "describe", "detail", "did","do", "done", "down", "due", "during","each", 
            "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", 
            "ever", "every","everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", 
            "fill", "find", "fire", "first","five", "for", "former", "formerly", "forty", "found", "four", 
            "from", "front", "full", "further", "get", "give","go", "had", "has", "hasnt", "have", "he", "hence", 
            "her", "here", "hereafter", "hereby", "herein", "hereupon","hers", "herself", "him", "himself", "his",
            "how", "however", "hundred", "i","ie", "if", "in", "inc", "indeed","interest", "into", "is", "it", "its", 
            "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd","made", "many", "may", "me",
            "meanwhile", "might", "mill", "mine", "miss", "more", "moreover", "most", "mostly", "move", "mr","mr.", "mrs","mrs.","much", "must", "my", 
            "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no","nobody", "none",
            "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one","only", 
            "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part","per", 
            "perhaps", "please", "put", "rather", "re","said", "same", "see", "seem", "seemed", "seeming", "seems", "serious",
            "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", 
            "somehow","someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system",
            "take", "ten", "than","that", "the", "their", "them", "themselves", "then", "thence", "there", 
            "thereafter", "thereby", "therefore","therein", "thereupon", "these", "they", "thickv", "thin", "third", 
            "this", "those", "though", "three", "through","throughout", "thru", "thus", "to", "together", "too", 
            "top", "toward", "towards", "twelve", "twenty", "two","un", "under", "until", "up", "upon", "us", "very",
            "via", "was", "we", "well", "were", "what", "whatever","when", "whence", "whenever", "where", "whereafter",
            "whereas", "whereby", "wherein", "whereupon", "wherever","whether", "which", "while", "whither", "who",
            "whoever", "whole", "whom", "whose", "why", "will", "with","within", "without", "would", "yet", 
            "you", "your", "yours", "yourself", "yourselves", "the", '', 'whatsoever', 'away', 'lady', 'ebook', 
            'www', 'online', 'org', 's', 'project', 'shall', 'sir', 'gutenberg', 'know', '1']

In [118]:
def split_words_from_book(b):
    words = re.split("\\W+", b.lower())
    words = filter(lambda x : x not in stopwords, words)
    return words

In [119]:
books = books.map(lambda x: (x[1], x[0]))
books = books.mapValues(lambda path: path.rsplit('/', 1)[-1])
books = books.map(lambda x: (x[1], x[0]))
D = books.count()
print(D)

8


In [120]:
tokenized_books = books.mapValues(lambda book: split_words_from_book(book))
words_from_book = tokenized_books.flatMapValues(lambda x: x).persist()

In [121]:
term_frecuencies = words_from_book.map(lambda x: (x,1))
term_frecuencies = term_frecuencies.reduceByKey(lambda v,w: v + w).persist()
term_frecuencies.take(10)

[(('emma.txt', 'emma10'), 1),
 (('emma.txt', 'jane'), 301),
 (('emma.txt', '1994'), 1),
 (('emma.txt', '158'), 1),
 (('emma.txt', 'public'), 14),
 (('emma.txt', 'domain'), 1),
 (('emma.txt', 'chapter'), 56),
 (('emma.txt', 'comfortable'), 34),
 (('emma.txt', 'home'), 132),
 (('emma.txt', 'unite'), 3)]

In [122]:
max_frecuencies = term_frecuencies
max_frecuencies = term_frecuencies.map(lambda x: (x[0][0], x[1])).reduceByKey(max)#.collect()
max_frecuencies.take(8)

[('emma.txt', 865),
 ('persuasion.txt', 497),
 ('lady_susan.txt', 110),
 ('mansfield_park.txt', 925),
 ('pride_and_prejudice.txt', 635),
 ('sense_and_sensibility.txt', 685),
 ('northanger abbey.txt', 487),
 ('love_and_friendship.txt', 80)]

In [123]:
def organize_terms(book_term):
    title = book_term[0][0]
    word = book_term[0][1]
    count_value = book_term[1]
    return Row(book=title,term=word,value=count_value)

def organize_max_tuple(max_values):
    title = max_values[0]
    max_value = max_values[1]
    return Row(book=title,value=max_value)

In [124]:
term_count_RDD = term_frecuencies.map(organize_terms)
max_count_RDD = max_frecuencies.map(organize_max_values)

term_count = sqlContext.createDataFrame(term_count_RDD).persist()
max_count = sqlContext.createDataFrame(max_count_RDD).persist()

In [125]:
tf = term_count.join(max_count, term_count.book == max_count.book)\
                .select(term_count.book, "term", (term_count.value/max_count.value).alias("frecuency"))

tf_query = tf.select('book', 'term', 'frecuency')
tf_query.show(5)

+--------------+------------+-------------------+
|          book|        term|          frecuency|
+--------------+------------+-------------------+
|lady_susan.txt|        jane|0.03636363636363636|
|lady_susan.txt|         use|0.12727272727272726|
|lady_susan.txt|restrictions|0.01818181818181818|
|lady_susan.txt|       title|0.01818181818181818|
|lady_susan.txt|     posting|0.00909090909090909|
+--------------+------------+-------------------+
only showing top 5 rows



In [126]:
document_frecuency_RDD = words_from_book.distinct().map(lambda x: (x[1],1)).reduceByKey(lambda v,w: v + w).persist()
document_frecuency_RDD.take(10)
# list(document_frecuency)[:10]

[('emma10', 1),
 ('jane', 8),
 ('1994', 5),
 ('158', 1),
 ('public', 8),
 ('domain', 7),
 ('chapter', 6),
 ('comfortable', 8),
 ('home', 8),
 ('unite', 7)]

In [127]:
def organize_idf(term):
    term_name = term[0]
    doc_count = term[1]
    if(doc_count == 0):
        term_idf = 0
    else:
        term_idf = log(D/doc_count)
    return Row(term=term_name,value=term_idf)

term_idf = document_frecuency_RDD.map(organize_doc_frecuency)
idf = sqlContext.createDataFrame(term_idf).persist()

In [128]:
tf_idf = tf.join(idf, tf.term == idf.term).select(tf.term, (tf.frecuency*idf.value).alias('tf-idf'), tf.book)
tf_idf_query = tf_idf.select('term', 'tf-idf', 'book').orderBy(desc('tf-idf'))
tf_idf_query.show(20)

+---------+-----------------+--------------------+
|     term|           tf-idf|                book|
+---------+-----------------+--------------------+
|     dear|              8.0|love_and_friendsh...|
|   father|              6.7|love_and_friendsh...|
|   mother|              6.4|love_and_friendsh...|
|   letter|              6.2|love_and_friendsh...|
|     love|              6.0|love_and_friendsh...|
|     anne|              6.0|      persuasion.txt|
|     time|              5.9|love_and_friendsh...|
|   having|              5.2|love_and_friendsh...|
|catherine|              5.0|northanger abbey.txt|
|elizabeth|              5.0|pride_and_prejudi...|
|     work|              5.0|love_and_friendsh...|
|  replied|              4.7|love_and_friendsh...|
|    think|              4.7|love_and_friendsh...|
|   mother|4.581818181818182|      lady_susan.txt|
|    great|              4.3|love_and_friendsh...|
|     dear|4.218181818181818|      lady_susan.txt|
|    young|              4.1|lo